In [21]:
import pandas as pd
import torch
from preprocessing import *
from PC_transformer import PCTransformer

set_seed(42)

In [22]:
df = pd.read_csv(r'output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [23]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,1.0,0.0,6680.0,7560.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
1,2.0,7.0,14297.0,7568.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,3.0,1590.0,17322.0,7561.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,4.0,2205.0,61.0,14972.0,8192.0,-1.0,-1.0,8192.0,21600.0,-1.0,-1.0,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0
4,5.0,2566.0,17357.0,7571.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0


In [24]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 20   # Number of training epochs
dropout = 0.2    # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)

In [25]:
# Create model
model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)

# Train the model
model.train_model(model, dataloaders, num_epochs, device, scaler)

# Final evaluation on test set
test_rmse, test_mae, test_mse, test_r2 = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)
print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}')

Epoch [1/20]
Train Loss: 0.2628, Val Loss: 0.3077
Val RMSE: 10679.1290, Val MAE: 5712.2331, Val MSE: 114043796.5924, Val R2: -0.0610
Epoch [2/20]
Train Loss: 0.1905, Val Loss: 0.1826
Val RMSE: 7269.8048, Val MAE: 3947.2241, Val MSE: 52850062.4339, Val R2: 0.5083
Epoch [3/20]
Train Loss: 0.1274, Val Loss: 0.1625
Val RMSE: 6857.8956, Val MAE: 3560.9478, Val MSE: 47030732.7342, Val R2: 0.5625
Epoch [4/20]
Train Loss: 0.1117, Val Loss: 0.1532
Val RMSE: 6829.1605, Val MAE: 3288.9219, Val MSE: 46637432.4839, Val R2: 0.5661
Epoch [5/20]
Train Loss: 0.1070, Val Loss: 0.1495
Val RMSE: 6788.8728, Val MAE: 3231.9042, Val MSE: 46088794.5525, Val R2: 0.5712
Epoch [6/20]
Train Loss: 0.1043, Val Loss: 0.1423
Val RMSE: 6634.0359, Val MAE: 3293.7281, Val MSE: 44010432.8323, Val R2: 0.5906
Epoch [7/20]
Train Loss: 0.1024, Val Loss: 0.1440
Val RMSE: 6835.2339, Val MAE: 3162.7783, Val MSE: 46720422.7881, Val R2: 0.5653
Epoch [8/20]
Train Loss: 0.1008, Val Loss: 0.1436
Val RMSE: 6716.6600, Val MAE: 3083.84